# The Battle of Neighborhoods Week 2

#### Import Librares

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

## Mapping Data

#### Current Resident Downtown LA "The Santee Alley"

In [9]:
neighborhood_latitude= 34.052235
neighborhood_longitude=-118.243683

In [7]:
# The code was removed by Watson Studio for sharing.

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={O3QICJOE3ZBSKBHDPUMJBWH35KLUAQWUPLFXSPYUV0VPVJJ5}&client_secret={PNFMBHNPUDHC5MSKVH5NYCDYAYBIOTUG0LXKKGXUCWQHILZB}&v={20170819}&ll={34.052235},{118.243683}&radius={500}&limit={100}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Map of Los angles 

In [18]:
import folium # map rendering library
import folium # map rendering library
from folium import plugins

In [ ]:
latitude=34.052235
longitude=-118.243683

map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

## BROOKLYN NEIGHBORHOODS - DATA AND MAPPING

In [24]:
import pandas as pd

In [25]:
brooklyn_data  = pd.read_csv('br_neigh_data.csv') 
brooklyn_data.head()

,Station,Unnamed: 1,Division,Line,Services,Opened [8][9][10]
0,Fourth Avenue*[^ 4],NaN,B (IND),Culver Line,F ​ G,"October 7, 1933[10]"
1,Seventh Avenue,NaN,B (BMT),Brighton Line,B ​ Q,"August 1, 1920[9]"
2,Seventh Avenue,NaN,B (IND),Culver Line,F <F> ​ G,"October 7, 1933[10]"
3,Eighth Avenue,NaN,B (BMT),Sea Beach Line,N ​ W,"June 22, 1915[9]"
4,Ninth Avenue,NaN,B (BMT),West End Line,D,"June 24, 1916[9]"


In [26]:
brooklyn_data.tail()

,Station,Unnamed: 1,Division,Line,Services,Opened [8][9][10]
165,Van Siclen Avenue,NaN,A (IRT),New Lots Line,2 ​ 3 ​ 4 ​ 5,"October 16, 1922[8]"
166,West Eighth Street–New York Aquarium***,NaN,"B (BMT, IND)","Brighton Line, Culver Line",F <F> ​​ Q,"May 30, 1917[9]"
167,Wilson Avenue,NaN,B (BMT),Canarsie Line,L,"July 14, 1928[9]"
168,Winthrop Street,NaN,A (IRT),Nostrand Avenue Line,2 ​ 5,"August 23, 1920[8]"
169,York Street‡,NaN,B (IND),Sixth Avenue Line,F <F> ​,"April 9, 1936[10]"


## Brooklyn map 

In [28]:
latitude= 40.650002
longitude= -73.949997

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
     label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters

FileNotFoundError: [Errno 2] File brooklyn_data.csv does not exist: 'brooklyn_data.csv'